In [ ]:
!pip install tensorflow==2.10.0 #tensorflow 2.10.0 <-- 2.12.0 (버전 다운그레이드 필요)

# How many epochs needs to be interated to get the best result on DialoGPT-large model
# when finetuning with one million words of new dataset?
'''
[ChatGPT & BingChat] Based on the experiences shared in the GitHub repository of the Hugging-Face team,
fine-tuning the Microsoft DialoGPT-large model with one million words requires at least several hundred epochs,
typically around 2000 epochs using a batch size of 2, because the model has a large number of parameters (774 million),
or until the validation loss plateaus. For example:

train_dataset = ... # create or load your training dataset
batch_size = 2
model.compile(loss=model.compute_loss, optimizer="adam")
model.fit(train_dataset.batch(batch_size), 
          validation_data=train_dataset.batch(batch_size),
          batch_size=batch_size, 
          epochs=2000)
'''

In [1]:
# Microsoft/DialoGPT-large(file size = 3GB) supports multiple languages

import tensorflow as tf
# GPU에 할당(사용)되는 메모리 크기 제한
gpus = tf.config.experimental.list_physical_devices('GPU')
gpu_memory_limit=1024*14 # only allocate 1GB(x14) of memory on the gpus[0], i.e. first GPU
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=gpu_memory_limit)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print("GPU memory allocation(",gpu_memory_limit,")",
          "# of Physical GPU(",len(gpus),") # of Logical GPU(",len(logical_gpus),")")
  except RuntimeError as e:
    print(e) # Virtual devices must be set before GPUs have been initialized

# Print the current date and time in the format:
# "YYYY-MM-DD HH:MM:SS.microseconds"
import datetime
def print_current_datetime(text=""):
    datetime_string = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
    print("{} @ CDT({})".format(text,datetime_string))

from transformers import AutoTokenizer, TFAutoModelForCausalLM
import tensorflow as tf
from timeit import default_timer

model_path = "DialoGPT-large-finetuned-by-Microsoft-Keras"
learning_rate = 1e-5
batch_size = 1
epochs = 666

print_current_datetime()

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = TFAutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

# Load the text data
with open('한글성경(마침표있음)_정제후말뭉치_오십만단어.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Tokenize the text data
tokenized_text = tokenizer(text, return_tensors='tf')

print_current_datetime("Finetune the DialoGPT-large-by-Microsoft-Keras")

'''
The below code defines a training function for a neural network model,
which takes input_ids as input. The input_ids are truncated to a maximum sequence length of 1024.
The function then computes the outputs and logits of the model based on the truncated input_ids.
The labels are extracted from input_ids starting from the second token.

logits = outputs.logits[:, :-1, :] # This line selects the logits for all tokens except the last one,
                                   # as the last token does not have a corresponding label.
labels = input_ids[:, 1:] # This line selects the labels for all tokens except the first one,
                          # as the first token is used as the input and not as a label.

Finally, the function computes the loss based on the logits and labels, and then applies
the gradients to the trainable weights of the model using the optimizer.
The function returns the loss_value as the output.
'''

# Define the training function
@tf.function
def train_step(input_ids):
    # Truncate input sequence
    #max_seq_length = 1024 # "1024" for DialoGPT-large
    max_seq_length = 256 # for RESOURCE_EXHAUSTED(OOM)
    input_ids = input_ids[:, :max_seq_length]
    with tf.GradientTape() as tape:
        outputs = model(input_ids, training=True)
        logits = outputs.logits[:, :-1, :]
        labels = input_ids[:, 1:]
        loss_value = loss(labels, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value

# Create a TensorSliceDataset from the tokenized text
dataset = tf.data.Dataset.from_tensor_slices(tokenized_text['input_ids'])
dataset = dataset.batch(batch_size)

# Define the loss function and optimizer
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Fine-tune the model
for epoch in range(epochs):
    start = default_timer()
    print(f'Epoch {epoch+1}/{epochs}')
    for step, batch in enumerate(dataset):
        loss_value = train_step(batch)
        if step % 50 == 0:
            print(f'Step {step} Loss {loss_value}')
    end = default_timer()
    print("Time duration(in seconds):", end - start)

print_current_datetime("Saving Fine-tuned Microsoft DialoGPT Model")

tokenizer.save_pretrained(model_path)
model.save_pretrained(model_path)

2023-05-23 08:42:33.173469: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 08:42:33.310552: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-23 08:42:33.348468: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-23 08:42:34.045799: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

GPU memory allocation( 14336 ) # of Physical GPU( 1 ) # of Logical GPU( 1 )
 @ CDT(2023-05-23T08:42:36.402268)


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at microsoft/DialoGPT-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (4749919 > 1024). Running this sequence through the model will result in indexing errors


Finetune the DialoGPT-large-by-Microsoft-Keras @ CDT(2023-05-23T08:42:49.058485)
Epoch 1/666
Step 0 Loss 19.62095832824707
Time duration(in seconds): 28.133458416094072
Epoch 2/666
Step 0 Loss 15.615325927734375
Time duration(in seconds): 0.47740102908574045
Epoch 3/666
Step 0 Loss 12.063688278198242
Time duration(in seconds): 0.4841521790949628
Epoch 4/666
Step 0 Loss 11.850034713745117
Time duration(in seconds): 0.48240926500875503
Epoch 5/666
Step 0 Loss 9.283498764038086
Time duration(in seconds): 0.48225169198121876
Epoch 6/666
Step 0 Loss 8.181551933288574
Time duration(in seconds): 0.4799278260907158
Epoch 7/666
Step 0 Loss 7.18070125579834
Time duration(in seconds): 0.4850624590180814
Epoch 8/666
Step 0 Loss 8.277751922607422
Time duration(in seconds): 0.48338681005407125
Epoch 9/666
Step 0 Loss 5.6581645011901855
Time duration(in seconds): 0.48259101202711463
Epoch 10/666
Step 0 Loss 5.3072333335876465
Time duration(in seconds): 0.48072359908837825
Epoch 11/666
Step 0 Loss 5.4

Step 0 Loss 1.7813308238983154
Time duration(in seconds): 0.49212229694239795
Epoch 93/666
Step 0 Loss 1.8421630859375
Time duration(in seconds): 0.4954056399874389
Epoch 94/666
Step 0 Loss 1.605406641960144
Time duration(in seconds): 0.49305689195171
Epoch 95/666
Step 0 Loss 2.1466009616851807
Time duration(in seconds): 0.49596515705343336
Epoch 96/666
Step 0 Loss 1.7613517045974731
Time duration(in seconds): 0.49703664402477443
Epoch 97/666
Step 0 Loss 1.6991668939590454
Time duration(in seconds): 0.4950610150117427
Epoch 98/666
Step 0 Loss 1.6768333911895752
Time duration(in seconds): 0.4956883699633181
Epoch 99/666
Step 0 Loss 1.7109687328338623
Time duration(in seconds): 0.4960929730441421
Epoch 100/666
Step 0 Loss 1.4863187074661255
Time duration(in seconds): 0.49296038900502026
Epoch 101/666
Step 0 Loss 1.7427763938903809
Time duration(in seconds): 0.49476304196286947
Epoch 102/666
Step 0 Loss 2.073702335357666
Time duration(in seconds): 0.4946333630941808
Epoch 103/666
Step 0 L

...

Step 0 Loss 0.23556482791900635
Time duration(in seconds): 0.6207432090304792
Epoch 274/666
Step 0 Loss 0.23985110223293304
Time duration(in seconds): 0.5076181549811736
Epoch 275/666
Step 0 Loss 0.23738262057304382
Time duration(in seconds): 0.518528921995312
Epoch 276/666
Step 0 Loss 0.1949211061000824
Time duration(in seconds): 0.5067491289228201
Epoch 277/666
Step 0 Loss 0.28619134426116943
Time duration(in seconds): 0.5134799600346014
Epoch 278/666
Step 0 Loss 0.18174989521503448
Time duration(in seconds): 0.5086373609956354
Epoch 279/666
Step 0 Loss 0.2230938822031021
Time duration(in seconds): 0.5133033930324018
Epoch 280/666
Step 0 Loss 0.182225301861763
Time duration(in seconds): 0.5080706910230219
Epoch 281/666
Step 0 Loss 0.21354469656944275
Time duration(in seconds): 0.5140545800095424
Epoch 282/666
Step 0 Loss 0.19775144755840302
Time duration(in seconds): 0.509326106053777
Epoch 283/666
Step 0 Loss 0.19827263057231903
Time duration(in seconds): 0.5120418780716136
Epoch 28

Step 0 Loss 0.10823062062263489
Time duration(in seconds): 0.5089072929695249
Epoch 364/666
Step 0 Loss 0.11534598469734192
Time duration(in seconds): 0.5099650560878217
Epoch 365/666
Step 0 Loss 0.0853264257311821
Time duration(in seconds): 0.5099937489721924
Epoch 366/666
Step 0 Loss 0.11841107159852982
Time duration(in seconds): 0.5132977199973539
Epoch 367/666
Step 0 Loss 0.10528188943862915
Time duration(in seconds): 0.5128908690530807
Epoch 368/666
Step 0 Loss 0.11170288920402527
Time duration(in seconds): 0.5091162270400673
Epoch 369/666
Step 0 Loss 0.1055716723203659
Time duration(in seconds): 0.5097604910843074
Epoch 370/666
Step 0 Loss 0.14380498230457306
Time duration(in seconds): 0.5134280070196837
Epoch 371/666
Step 0 Loss 0.06869401782751083
Time duration(in seconds): 0.513528395909816
Epoch 372/666
Step 0 Loss 0.1012144610285759
Time duration(in seconds): 0.5110764190321788
Epoch 373/666
Step 0 Loss 0.06588876992464066
Time duration(in seconds): 0.5139606388984248
Epoch 

Step 0 Loss 0.0416509173810482
Time duration(in seconds): 0.5116913509555161
Epoch 454/666
Step 0 Loss 0.07782138884067535
Time duration(in seconds): 0.5105092469602823
Epoch 455/666
Step 0 Loss 0.06666299700737
Time duration(in seconds): 0.510055071907118
Epoch 456/666
Step 0 Loss 0.05229060724377632
Time duration(in seconds): 0.5105417378945276
Epoch 457/666
Step 0 Loss 0.08035612106323242
Time duration(in seconds): 0.5105832970002666
Epoch 458/666
Step 0 Loss 0.05193148925900459
Time duration(in seconds): 0.5098096650326625
Epoch 459/666
Step 0 Loss 0.07260704785585403
Time duration(in seconds): 0.5058109140954912
Epoch 460/666
Step 0 Loss 0.058151938021183014
Time duration(in seconds): 0.5116889129858464
Epoch 461/666
Step 0 Loss 0.07693324238061905
Time duration(in seconds): 0.5079448540927842
Epoch 462/666
Step 0 Loss 0.04200468584895134
Time duration(in seconds): 0.5106724709039554
Epoch 463/666
Step 0 Loss 0.07310621440410614
Time duration(in seconds): 0.510688891983591
Epoch 4

Step 0 Loss 0.014831927604973316
Time duration(in seconds): 0.5101068529766053
Epoch 544/666
Step 0 Loss 0.07650040090084076
Time duration(in seconds): 0.5101932039251551
Epoch 545/666
Step 0 Loss 0.06761249899864197
Time duration(in seconds): 0.5119225160451606
Epoch 546/666
Step 0 Loss 0.05122093856334686
Time duration(in seconds): 0.5090220510028303
Epoch 547/666
Step 0 Loss 0.06218433752655983
Time duration(in seconds): 0.5086541190976277
Epoch 548/666
Step 0 Loss 0.023839149624109268
Time duration(in seconds): 0.5097918009851128
Epoch 549/666
Step 0 Loss 0.03903166949748993
Time duration(in seconds): 0.5071449390379712
Epoch 550/666
Step 0 Loss 0.033532410860061646
Time duration(in seconds): 0.5088579420698807
Epoch 551/666
Step 0 Loss 0.04680347442626953
Time duration(in seconds): 0.5104310919996351
Epoch 552/666
Step 0 Loss 0.02808935008943081
Time duration(in seconds): 0.5124577300157398
Epoch 553/666
Step 0 Loss 0.03612354025244713
Time duration(in seconds): 0.5092692399630323

Step 0 Loss 0.05392438918352127
Time duration(in seconds): 0.5124911710154265
Epoch 633/666
Step 0 Loss 0.007819102145731449
Time duration(in seconds): 0.5091155810514465
Epoch 634/666
Step 0 Loss 0.018728341907262802
Time duration(in seconds): 0.5088927350006998
Epoch 635/666
Step 0 Loss 0.009961448609828949
Time duration(in seconds): 0.5074017579900101
Epoch 636/666
Step 0 Loss 0.0057260747998952866
Time duration(in seconds): 0.5090248709311709
Epoch 637/666
Step 0 Loss 0.008822870440781116
Time duration(in seconds): 0.5079971350496635
Epoch 638/666
Step 0 Loss 0.03855396807193756
Time duration(in seconds): 0.5084381490014493
Epoch 639/666
Step 0 Loss 0.03438541665673256
Time duration(in seconds): 0.5106362180085853
Epoch 640/666
Step 0 Loss 0.054499123245477676
Time duration(in seconds): 0.5086356080137193
Epoch 641/666
Step 0 Loss 0.011517602019011974
Time duration(in seconds): 0.5090497779892758
Epoch 642/666
Step 0 Loss 0.018095681443810463
Time duration(in seconds): 0.5064341221

In [1]:
# Print the current date and time in the format:
# "YYYY-MM-DD HH:MM:SS.microseconds"
import datetime
def print_current_datetime(text=""):
    datetime_string = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
    print("{} @ CDT({})".format(text,datetime_string))

from transformers import AutoTokenizer, TFAutoModelForCausalLM
from timeit import default_timer

model_path = "DialoGPT-large-finetuned-by-Microsoft-Keras"

print_current_datetime("Loading Fine-tuned Microsoft DialoGPT Model")

tokenizer_loaded = AutoTokenizer.from_pretrained(model_path)
model_loaded = TFAutoModelForCausalLM.from_pretrained(model_path)

print_current_datetime()

def generate_ouput(prompt=""):

    start = default_timer()

    input_ids = tokenizer_loaded.encode(prompt, return_tensors="tf")
    output_ids = model_loaded.generate(input_ids=input_ids,
                                       max_length=1024+input_ids.shape[1],
                                       temperature=0.7,
                                       top_p=0.9,
                                       do_sample=True,
                                       num_return_sequences=5, # The model will generate five different responses to the prompt.
                                       pad_token_id=tokenizer_loaded.eos_token_id)
    generated_text = tokenizer_loaded.decode(output_ids[0], skip_special_tokens=True)

    end = default_timer()

    # num_return_sequences=5, which means the model will generate 5 different responses to the prompt.
    # The below code loops through the generated responses and print them out with a response number.
    for i, return_sequence in enumerate(output_ids):
        print(f'Response {i+1}: {tokenizer_loaded.decode(return_sequence, skip_special_tokens=True)}')

    print("Time duration(in seconds):", end - start)
    return generated_text

# Let's chat for 10 lines
for step in range(10):
    prompt = input(">> User:")
    if prompt.lower() == "bye": break

    generated_text = generate_ouput(prompt)
    split_generated_text = generated_text.split(prompt)
    if len(split_generated_text) > 1:
        generated_text = split_generated_text[1]
    # Trim the sentences after the last period(.)
    text_to_remove = generated_text.split('.')[-1]
    generated_text = generated_text.replace(text_to_remove,'')

    print(">> GPT: {}".format( generated_text ))
    print_current_datetime()

Loading Fine-tuned Microsoft DialoGPT Model @ CDT(2023-05-23T10:34:31.564298)


2023-05-23 10:34:31.662721: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 10:34:31.974579: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-23 10:34:32.051553: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-23 10:34:32.783001: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

 @ CDT(2023-05-23T10:34:41.922718)
>> User:Jesus
Response 1: Jesus� sama
Response 2: Jesus�s! Jesus Christ!
Response 3: Jesus� is not a thing. Jesus Christophobe. Christophobe Christophobe. Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christique Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christophobe Christ

>> User:bye
